In [2]:
import pandas as pd

liste_dico = [{"match_drop" : [1550555, 1546206],
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"match_drop" : [],
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"},
           {"match_drop" : [129364, 128946],
                "ranking" : ["Toulouse FC", "AC Ajaccio", "AJ Auxerre", "Paris FC", "FC Sochaux-Montbéliard", "En Avant de Guingamp",
                             "SM Caen", "Le Havre AC", "Nîmes Olympique", "Pau FC", "Dijon FCO", "SC Bastia", "Chamois Niortais FC", 
                             "Amiens Sporting Club", "Grenoble Foot 38", "Valenciennes FC", "Rodez Aveyron", "US Quevilly-Rouen",
                             "Dunkerque", "AS Nancy-Lorraine"],
           "annee" : "2021_2022"}
           ]
dico = liste_dico[0]
def rename_col(col) :
    if "full_all" in col :
        return col.replace("full_all", "per_Match")
    elif "full_otip_p30otip" in col :
        return col.replace("full_otip_p30otip", "per30otip")
    elif "full_tip_p30tip" in col :
        return col.replace("full_tip_p30tip", "per30tip")
    else :
        return col

In [3]:
data_import = pd.read_excel(f"../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/data_physical.xlsx", index_col = 0)

In [5]:
data = data_import[~(data_import.match_id.isin(dico["match_drop"]))]
data.fillna(0, inplace = True)

drop = ["player_name", "player_short_name", "psv99", "player_id", "player_birthdate", "team_id", "match_name", "match_date", "competition_name", "competition_id", "season_name",
        "season_id", "competition_edition_id", "position", "position_group", "minutes_full_tip", "minutes_full_otip", "physical_check_passed"]

data.drop(drop, inplace = True, axis = 1)
sample = data.columns[["sample" in i for i in data.columns]]
data.drop(sample, inplace = True, axis = 1)

data.rename(columns = rename_col, inplace = True)

# AJOUT NUMÉRO DE JOURNÉE AU DATAFRAME
match_round = pd.read_excel(f"../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/match_round.xlsx",
                            index_col = 0)
match_round.drop(index = dico["match_drop"], inplace = True)
data = data.merge(match_round, right_on = "id", left_on = "match_id").drop("match_id", axis = 1)
data = data.groupby(["Journée", "team_name"], as_index = True)
nb_joueur_match = data.apply(len)
data = data.sum()
data = data.divide(nb_joueur_match, axis = 0)

nb_minute_match = data.pop("minutes_per_Match")
data = data.multiply(900/nb_minute_match, axis = 0).reindex(dico["ranking"], axis = 0, level = 1)

C:\Users\fabie\AppData\Local\Temp\ipykernel_29540\1217641683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna(0, inplace = True)
C:\Users\fabie\AppData\Local\Temp\ipykernel_29540\1217641683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(drop, inplace = True, axis = 1)
C:\Users\fabie\AppData\Local\Temp\ipykernel_29540\1217641683.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(sample, inplace =

In [8]:
info = pd.read_excel("../../Info matchs/Skill Corner/result_2023_2024.xlsx", index_col = [0, 1])

In [15]:
pd.merge(info, data, on = ["Journée", "team_name"])

result  total_distance_per_Match  \
Journée team_name                                            
1       AJ Auxerre           win              97741.190814   
        Angers SCO          lose              94935.039592   
        AS Saint-Étienne    lose              97790.091009   
        Rodez Aveyron       draw              95545.247831   
        Paris FC            lose             100340.985382   
...                          ...                       ...   
38      Dunkerque           draw             101896.774329   
        ES Troyes AC        draw              99078.583731   
        US Quevilly-Rouen    win             104547.212309   
        US Concarneau       lose             106056.716418   
        Valenciennes FC     lose             107729.683576   

                           total_metersperminute_per_Match  \
Journée team_name                                            
1       AJ Auxerre                             1518.099055   
        Angers SCO                             1448.511303   
        AS Saint-Étienne                       1439.383924   
        Rodez Aveyron                          1383.276646   
        Paris FC                               1528.547916   
...                                                    ...   
38      Dunkerque                              1539.230793   
        ES Troyes AC                           1418.661633   
        US Quevilly-Rouen                      1669.310199   
        US Concarneau                          1801.084577   
        Valenciennes FC                        1505.879458   

                           running_distance_per_Match  hsr_distance_per_Match  \
Journée team_name                                                               
1       AJ Auxerre                       14333.839666             5306.497712   
        Angers SCO                       13432.477123             4870.335945   
        AS Saint-Étienne                 14100.011768             5671.779382   
        Rodez Aveyron                    13177.379906             4928.727023   
        Paris FC                         13731.456416             5256.740661   
...                                               ...                     ...   
38      Dunkerque                        13450.827674             5199.701890   
        ES Troyes AC                     13589.938719             5442.283815   
        US Quevilly-Rouen                14847.116660             5711.141508   
        US Concarneau                    15770.149254             5776.119403   
        Valenciennes FC                  18091.310899             6512.004018   

                           hsr_count_per_Match  sprint_distance_per_Match  \
Journée team_name                                                           
1       AJ Auxerre                  509.963424                1859.972418   
        Angers SCO                  493.596393                1851.448643   
        AS Saint-Étienne            517.221089                1464.282912   
        Rodez Aveyron               489.261283                1672.430924   
        Paris FC                    503.844071                1540.768814   
...                                        ...                        ...   
38      Dunkerque                   518.647590                1305.593752   
        ES Troyes AC                540.713742                1960.087315   
        US Quevilly-Rouen           573.891043                2088.222892   
        US Concarneau               538.308458                1728.358209   
        Valenciennes FC             564.138624                1939.226519   

                           sprint_count_per_Match  hi_distance_per_Match  \
Journée team_name                                                          
1       AJ Auxerre                      91.739452            7166.470130   
        Angers SCO                      95.206795            6721.784588   
        AS Saint-Étienne                92.676134   

In [7]:
data.groupby("Journée").apply(len)

Journée
1     19
2     16
3     16
4     20
5     18
6     18
7     20
8     20
9     20
10    20
11    20
12    18
13    20
14    20
15    20
16    18
17    20
18    20
19    20
20    20
21    20
22    20
23    20
24    20
25    20
26    20
27    20
28    20
29    18
30    20
31    20
32    20
33    20
34    20
35    20
36    18
37    20
38    20
dtype: int64

In [16]:
data

result  \
Journée team_name                           
1       AC Ajaccio                   draw   
        AJ Auxerre                    win   
        AS Saint-Étienne             lose   
        Amiens Sporting Club          win   
        Angers SCO                   lose   
...                                   ...   
38      SM Caen                       win   
        Stade Lavallois Mayenne FC    win   
        US Concarneau                lose   
        US Quevilly-Rouen             win   
        Valenciennes FC              lose   

                                    count_opportunities_to_pass_to_runs_in_behind_per_match  \
Journée team_name                                                                             
1       AC Ajaccio                                                          24.935512         
        AJ Auxerre                                                          34.177443         
        AS Saint-Étienne                                                    33.539934         
        Amiens Sporting Club                                                30.187719         
        Angers SCO                                                          39.746526         
...                                                                               ...         
38      SM Caen                                                             25.313913         
        Stade Lavallois Mayenne FC                                          19.691696         
        US Concarneau                                                       16.915423         
        US Quevilly-Rouen                                                   20.363876         
        Valenciennes FC                                                     24.409844         

                                    count_pass_attempts_to_runs_in_behind_per_match  \
Journée team_name                                                                     
1       AC Ajaccio                                                        11.177988   
        AJ Auxerre                                                        12.591689   
        AS Saint-Étienne                                                  13.239448   
        Amiens Sporting Club                                              19.778161   
        Angers SCO                                                        25.881459   
...                                                                             ...   
38      SM Caen                                                           14.465093   
        Stade Lavallois Mayenne FC                                        13.426156   
        US Concarneau                                                      8.955224   
        US Quevilly-Rouen                                                 10.181938   
        Valenciennes FC                                                    9.040683   

                                    pass_opportunities_to_runs_in_behind_threat_per_match  \
Journée team_name                                                                           
1       AC Ajaccio                                                           0.739467       
        AJ Auxerre                                                           1.250175       
        AS Saint-Étienne                                                     0.917935       
        Amiens Sporting Club                                                 0.582935       
        Angers SCO                                                           0.785687       
...                                                                               ...       
38      SM Caen                                                              0.759417       
        Stade Lavallois Mayenne FC                                           0.617603       
        US Concarneau                                                        0.477612       
        US Quevilly-Rouen                                 

In [123]:
data.groupby("team_name").apply(len).reindex(dico["ranking"])

team_name
AJ Auxerre                    36
Angers SCO                    38
AS Saint-Étienne              38
Rodez Aveyron                 38
Paris FC                      39
SM Caen                       38
Stade Lavallois Mayenne FC    36
Amiens Sporting Club          38
En Avant de Guingamp          38
Pau FC                        37
Grenoble Foot 38              37
Girondins de Bordeaux         37
SC Bastia                     37
FC Annecy                     38
AC Ajaccio                    38
Dunkerque                     38
ES Troyes AC                  37
US Quevilly-Rouen             38
US Concarneau                 38
Valenciennes FC               36
dtype: int64

In [117]:
data.index.get_level_values(0)

Index(['AC Ajaccio', 'AJ Auxerre', 'AS Saint-Étienne', 'Amiens Sporting Club',
       'Angers SCO', 'Dunkerque', 'ES Troyes AC', 'En Avant de Guingamp',
       'FC Annecy', 'Girondins de Bordeaux', 'Grenoble Foot 38', 'Paris FC',
       'Pau FC', 'Rodez Aveyron', 'SC Bastia', 'SM Caen',
       'Stade Lavallois Mayenne FC', 'US Concarneau', 'US Quevilly-Rouen',
       'Valenciennes FC'],
      dtype='object', name='team_name')

In [45]:
result = data[["match_id", "result"]].head(1)
result

,match_id,result
team_name,,
Rodez Aveyron,1007270,draw
Rodez Aveyron,1036536,lose
Rodez Aveyron,1050072,lose
Rodez Aveyron,1065099,draw
Rodez Aveyron,1093842,win
...,...,...
Girondins de Bordeaux,1516648,win
AC Ajaccio,1099624,lose
AC Ajaccio,1354154,lose


In [31]:
result

team_name
Rodez Aveyron            draw
Rodez Aveyron            lose
Rodez Aveyron            lose
Rodez Aveyron            draw
Rodez Aveyron             win
                         ... 
Girondins de Bordeaux     win
AC Ajaccio               lose
AC Ajaccio               lose
US Quevilly-Rouen         win
Pau FC                    win
Name: result, Length: 750, dtype: object

In [22]:
result1 = result.groupby(["team_name", "match_id"]).head(1)

In [24]:
result1.result

AttributeError: 'DataFrame' object has no attribute 'result'

In [29]:
def rename_col(col) :
    if "full_all" in col :
        return col.replace("full_all", "per_Match")
    elif "full_otip_p30otip" in col :
        return col.replace("full_otip_p30otip", "per30otip")
    elif "full_otip_p30tip" in col :
        return col.replace("full_otip_p30tip", "per30tip")
    else :
        return col

In [30]:
data.rename(columns = rename_col)

,match_id,minutes_per_Match,total_distance_per_Match,total_metersperminute_per_Match,running_distance_per_Match,hsr_distance_per_Match,hsr_count_per_Match,sprint_distance_per_Match,sprint_count_per_Match,hi_distance_per_Match,...,hsr_distance_per30otip,hsr_count_per30otip,sprint_distance_per30otip,sprint_count_per30otip,hi_distance_per30otip,hi_count_per30otip,medaccel_count_per30otip,highaccel_count_per30otip,meddecel_count_per30otip,highdecel_count_per30otip
team_name,,,,,,,,,,,,,,,,,,,,,
Angers SCO,1007272,97.37,9583.0,98.42,1230.0,355.0,39.0,145.0,8.0,500.0,...,378.03,34.89,79.97,4.36,458.00,39.26,65.43,4.36,40.71,7.27
Angers SCO,1021419,98.10,10355.0,105.56,1074.0,241.0,23.0,90.0,5.0,331.0,...,142.41,14.90,77.83,4.97,220.24,19.87,51.33,3.31,33.12,3.31
Angers SCO,1035154,102.35,10006.0,97.76,901.0,289.0,33.0,83.0,7.0,372.0,...,198.38,19.05,32.50,3.36,230.88,22.42,49.32,1.12,32.50,2.24
Angers SCO,1050074,99.27,10528.0,106.06,1054.0,273.0,27.0,60.0,4.0,333.0,...,231.11,20.00,36.67,1.11,267.78,21.11,54.44,2.22,35.56,4.44
Angers SCO,1064337,99.68,9441.0,94.71,876.0,186.0,22.0,34.0,3.0,220.0,...,149.13,14.60,16.69,2.09,165.82,16.69,51.10,2.09,33.37,6.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Girondins de Bordeaux,1543780,26.77,3076.0,114.92,436.0,139.0,19.0,57.0,3.0,196.0,...,364.93,54.25,152.88,4.93,517.81,59.18,49.32,0.00,39.45,9.86
Girondins de Bordeaux,1527200,10.67,1261.0,118.22,187.0,123.0,6.0,19.0,1.0,142.0,...,659.73,24.43,65.16,0.00,724.89,24.43,40.72,0.00,24.43,0.00
ES Troyes AC,1539379,12.60,1556.0,123.49,204.0,142.0,12.0,15.0,2.0,157.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [21]:
[j.replace("full_otip_", "") for j in data_col]

['total_distance_p30otip',
 'total_metersperminute_p30otip',
 'running_distance_p30otip',
 'hsr_distance_p30otip',
 'hsr_count_p30otip',
 'sprint_distance_p30otip',
 'sprint_count_p30otip',
 'hi_distance_p30otip',
 'hi_count_p30otip',
 'medaccel_count_p30otip',
 'highaccel_count_p30otip',
 'meddecel_count_p30otip',
 'highdecel_count_p30otip']

In [22]:
data.columns[["full_otip_" in i for i in data.columns]] = [j.replace("full_otip_", "") for j in data_col]

TypeError: Index does not support mutable operations